### NOTES (Moved to Linux)
### PART I -- Start
This program is used to split combined RSQSim fault, neigh model, KZero and run model into individual faults model. 
First, combine all RSQSim file into CSV file and import CSV file in the program, need to specify from where fault model column and fault neigh column starts and end; number of fault models. 
- Can create folder & upload files for each individual faults
- updates on eq.out file name 
-Dates: 03-Feb-2022

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import numpy as np
from matplotlib.widgets import Cursor, Button
from collections import Counter         
from matplotlib import pyplot as plt
import re

Input Parameters

In [2]:
#Combined_Bruce_RSQSim_Parameters
output_folder_name = '3_Output'
input_file_location = '/home/govindlinux/PhD/RSQSim_file_prep/1_Input_Files'
fault_model_lower_column = 0                            # column from where RSQSim fault model input file starts
fault_model_higher_column = 19                          # End column
neighbour_model_lower_column = 25                       # column from where RSQSim neigh model input file starts
neighbour_model_higher_column = 400                     # End column
start_flt_no = 0
End_flt_no = 527

File input

In [3]:
'''df_fault_name & df_fault_name_1 are used to assign file name and to filter particular fault data respectively'''

df = pd.read_csv(r'%s/1_Combined_Bruce_RSQSim_Parameters.csv'%input_file_location)
df_KZero = pd.read_csv(r'%s/2_zfault_Deepen_KZero_GN.csv'%input_file_location)
df_run = pd.read_csv(r'%s/3_run.csv'%input_file_location)
df_fault_name = pd.read_csv(r'%s/4_znames_Deepen.in'%input_file_location)
df_fault_name_1 = pd.read_csv(r'%s/4_znames_Deepen.in'%input_file_location)
#dfgn = pd.read_csv(r'run.txt')
#dfgn.head()

Removing dublicates names and sepcial characters from data; only one character is removed at a time so there are 4 rows of 
simila coding.

In [4]:
'''as RSQSim fault, neigh, kzero and run file name do not accept space between letters, so this function is used to 
remove all special characters and dublicates file names; are used in all file names'''

df_fault_name.columns = ["fault name"]                                         # Creating column heading PD dataframe
df_fault_name.drop_duplicates(subset=None, keep='first', inplace=True)         # Removing dublicate name in columns                                 # inserting serial number 
df_fault_name = df_fault_name.replace("\['",'',regex=True).astype(str)         # Removing special character 
df_fault_name = df_fault_name.replace("\']",'',regex=True).astype(str)
df_fault_name = df_fault_name.replace("\ ",'',regex=True).astype(str)
df_fault_name = df_fault_name.replace("\-",'',regex=True).astype(str)
df_fault_name.head(2)

,fault name
0,Greendaless
303,AhuririRrv


In [5]:
import os
os.mkdir('/home/govindlinux/PhD/RSQSim_file_prep/%s'%(output_folder_name))

#### Import RSQSim run file

df.set_index('ids').filter(like='ball', axis=0)

data.iloc[0:5, 5:8] # first 5 rows and 5th, 6th, 7th columns of data frame 
data.iloc[0:5] # first five rows of dataframe
data.iloc[:, 0:2] # first two columns of data frame with all rows
data.iloc[[0,3,6,24], [0,5,6]] # 1st, 4th, 7th, 25th row + 1st 6th 7th columns.
data.iloc[0:5, 5:8] # first 5 rows and 5th, 6th, 7th columns of data frame (county -> phone1).

#'Hello, %s' % name
#"Hello, Bob"

#### Prepare fault, neigh, Kzero and run file

In [6]:
start_flt_no = start_flt_no
End_flt_no = End_flt_no

df_fault_name_1.columns=["fault name"]                                         # Creating column heading PD dataframe
df_fault_name_1.drop_duplicates(subset=None, keep='first', inplace=True)       # Removing dublicate name in columns  

# Initialize time bar (% completion bar)
from tqdm import tqdm
pbar = tqdm(desc='t',total=End_flt_no)

# While loop initilization
while start_flt_no <= End_flt_no:
    fault_name_1 = df_fault_name_1.iloc[start_flt_no]["fault name"]# Extracting eachrow of column-"fault name"fromfault_name_1
    fault_name = df_fault_name.iloc[start_flt_no]["fault name"]    # Extracting eachrow of column-"fault name" from fault_name
    
# Creating Folder for individual faults
    import os
    os.mkdir('/home/govindlinux/PhD/RSQSim_file_prep/%s/%d_%s'%(output_folder_name,start_flt_no,fault_name))
    
# Creating RSQSim Fault Model
    df_fault_filt = df.set_index('Fault Name').filter(like='%s'%fault_name_1, axis=0)  #Filter data of particular fault
    df_fault_filt_indvi = df_fault_filt.iloc[:,fault_model_lower_column:fault_model_higher_column] #clip all row & certain columns
    df_fault_filt_indvi.to_csv(r"/home/govindlinux/PhD/RSQSim_file_prep/%s/%d_%s/%d_%s_fault.txt"%(output_folder_name,start_flt_no,fault_name,start_flt_no,fault_name), header=None, index=None, sep=' ', mode='w')# export
    
# Creating RSQSim neigh model
    df_neigh_filt_indvi = df_fault_filt.iloc[:,neighbour_model_lower_column:neighbour_model_higher_column]#clip all row &column range
    df_neigh_filt_indvi.to_csv(r"/home/govindlinux/PhD/RSQSim_file_prep/%s/%d_%s/%d_%s_neigh.txt"%(output_folder_name,start_flt_no,fault_name,start_flt_no,fault_name), header=None, index=None, sep=' ', mode='w')#export
    
# Creating RSQSim KZero model
    df_fault_name['index_no'] = df_fault_name.index # adding index number in a column

# df_KZero = pd.read_csv(r'zfault_Deepen_KZero_GN.csv')
    df_KZero.columns = ["f1","f2"]  # adding columns heading "f1" & "f2" on df_KZero files
    start_flt_no_plus_1 = start_flt_no + 1
    a1 = df_fault_name.iloc[start_flt_no]['index_no']
    a2 = df_fault_name.iloc[start_flt_no_plus_1]['index_no']
    filter = ((df_KZero['f1']>= a1) & (df_KZero['f1']<=a2))
    df_KZero_1 = df_KZero[filter]
    df_KZero_1.to_csv(r"/home/govindlinux/PhD/RSQSim_file_prep/%s/%d_%s/%d_%s_Kzero.txt"%(output_folder_name,start_flt_no,fault_name,start_flt_no,fault_name), header=None, index=None, sep=' ', mode='w')

# Creating RSQSim RUN file
    df_run.columns = ["A","B","C"]   # Adding heading, although it is also in .csv file,  dont remove 
    df_run.iloc[31]["C"] = "%d_%s_fault.txt"%(start_flt_no,fault_name)  # adding fault name on 31st row in df_run file
    df_run.iloc[32]["C"] = "%d_%s"%(start_flt_no,fault_name)  # adding output file name on 32st row in df_run file
    df_run.iloc[65]["C"] = "%d_%s_Kzero.txt"%(start_flt_no,fault_name)  # adding fault name on 65 row in df_run file
    df_run.iloc[67]["C"] = "%d_%s_neigh.txt"%(start_flt_no,fault_name)  # adding fault name on 67 row in df_run file
    df_run.to_csv(r"/home/govindlinux/PhD/RSQSim_file_prep/%s/%d_%s/%d_%s_run.txt"%(output_folder_name, start_flt_no,fault_name,start_flt_no,fault_name), header=None, index=None, sep=' ',mode='w')
    

# Update for time bar
    pbar.update(n=1)
# Updating while loop
    start_flt_no = start_flt_no + 1  
# Close time bar
pbar.close()    

t: 528it [03:28,  2.53it/s]                                                                                             


In [7]:
print(a1)

272688


### PART I -- END

##############################################################################

### PART II -- Start
Import all RSQSim output file, combine them and export to CSV file

Date: 04_Feb_2022

In [8]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import numpy as np
from matplotlib.widgets import Cursor, Button
from collections import Counter         
from matplotlib import pyplot as plt
import re

#### Input Parameters

In [9]:
#Combined_Bruce_RSQSim_Parameters
output_folder_name = '3_RSQSim_Output_Faults_50-60'
#combined_eqs_output_name = '50-55_Faults'
input_file_location = '/home/govindlinux/PhD/RSQSim_file_prep/1_Input_Files'
fault_model_lower_column = 0                            # column from where RSQSim fault model input file starts
fault_model_higher_column = 19                          # End column
neighbour_model_lower_column = 25                       # column from where RSQSim neigh model input file starts
neighbour_model_higher_column = 400                     # End column
start_flt_no = 50
End_flt_no = 55

#### Input file 
names to prepare fault name. Without these files it is not possible to run program. Keep all these files into 1_Input_Files. 

In [10]:
'''df_fault_name & df_fault_name_1 are used to assign file name and to filter particular fault data respectively'''

df = pd.read_csv(r'%s/1_Combined_Bruce_RSQSim_Parameters.csv'%input_file_location)
df_KZero = pd.read_csv(r'%s/2_zfault_Deepen_KZero_GN.csv'%input_file_location)
df_run = pd.read_csv(r'%s/3_run.csv'%input_file_location)
df_fault_name = pd.read_csv(r'%s/4_znames_Deepen.in'%input_file_location)
df_fault_name_1 = pd.read_csv(r'%s/4_znames_Deepen.in'%input_file_location)

In [11]:
'''as RSQSim fault, neigh, kzero and run file name do not accept space between letters, so this function is used to 
remove all special characters and dublicates file names; are used in all file names'''

df_fault_name.columns = ["fault name"]                                         # Creating column heading PD dataframe
df_fault_name.drop_duplicates(subset=None, keep='first', inplace=True)         # Removing dublicate name in columns                                 # inserting serial number 
df_fault_name = df_fault_name.replace("\['",'',regex=True).astype(str)         # Removing special character 
df_fault_name = df_fault_name.replace("\']",'',regex=True).astype(str)
df_fault_name = df_fault_name.replace("\ ",'',regex=True).astype(str)
df_fault_name = df_fault_name.replace("\-",'',regex=True).astype(str)
df_fault_name.head(2)
fault_name = df_fault_name.iloc[start_flt_no]["fault name"]
print(fault_name)

Calypso02nv


#### 1st Fault output data

In [20]:
# Preparing files
df = pd.read_csv('/home/govindlinux/PhD/RSQSim_file_prep/%s/%d_%s/eqs.%d_%s.out'%(output_folder_name,start_flt_no,fault_name,start_flt_no,fault_name), header=None)
df.columns = ['A']    # Adding heading to dataframe
# First row
'''searching index number of rows having name as "%%%end input files" string'''
first_row_index = (df[df['A'].str.contains("%%% end input files")]).index.values  
first_row_index = first_row_index.astype(int)       #Converting index number to integer
''' this is used to extract a integer from array, dont confuse with .sum, this is not standard technique but it works'''
first_row_index = first_row_index[first_row_index < 50000].sum() 
# Last row
last_row_index = df.index   # Indexing row number
last_row_index = len(last_row_index)  # Obtaining length of index which is last index number
# Exporting files in CSV format    
df_fault = df.iloc[first_row_index+1:last_row_index,:] # clip all row above %%%end input files".
df_fault.to_csv(r'/home/govindlinux/PhD/RSQSim_file_prep/%s/%d_%s/eqs.%d_%s.csv'%(output_folder_name,start_flt_no,fault_name,start_flt_no,fault_name), header=None)

#### Combined Faults RSQSim  output data

In [21]:
df_initial = pd.read_csv('/home/govindlinux/PhD/RSQSim_file_prep/%s/%d_%s/eqs.%d_%s.csv'%(output_folder_name,start_flt_no,fault_name,start_flt_no,fault_name), header=None)
start_flt_no1 = start_flt_no
start_flt_no = start_flt_no + 1
end_flt_no = End_flt_no
df_fault_name_1.columns=["fault name"]                                         # Creating column heading PD dataframe
df_fault_name_1.drop_duplicates(subset=None, keep='first', inplace=True)       # Removing dublicate name in columns  

# Initialize time bar (% completion bar)
from tqdm import tqdm
pbar = tqdm(desc='t',total=end_flt_no-start_flt_no)

# While loop initilization
while start_flt_no <= end_flt_no:
    
#    fault_name_1 = df_fault_name_1.iloc[start_flt_no]["fault name"]# Extracting eachrow of column-"fault name"fromfault_name_1
    fault_name = df_fault_name.iloc[start_flt_no]["fault name"]    # Extracting eachrow of column-"fault name" from fault_name
# Preparing files
    df = pd.read_csv('/home/govindlinux/PhD/RSQSim_file_prep/%s/%d_%s/eqs.%d_%s.out'%(output_folder_name,start_flt_no,fault_name,start_flt_no,fault_name), header=None)
    df.columns = ['A']
# First row
    first_row_index = (df[df['A'].str.contains("%%% end input files")]).index.values
    first_row_index = first_row_index.astype(int)
    first_row_index = first_row_index[first_row_index < 5000000].sum()
# Last row
    last_row_index = df.index
    last_row_index = len(last_row_index)
# Exporting files in CSV format    
    df_fault = df.iloc[first_row_index+1:last_row_index,:] #clip all row & certain columns
    df_fault.to_csv(r'/home/govindlinux/PhD/RSQSim_file_prep/%s/%d_%s/eqs.%d_%s.csv'%(output_folder_name,start_flt_no,fault_name,start_flt_no,fault_name), header=None)
# Importing RSQSim output data
    df_intermediate = pd.read_csv('/home/govindlinux/PhD/RSQSim_file_prep/%s/%d_%s/eqs.%d_%s.csv'%(output_folder_name,start_flt_no,fault_name,start_flt_no,fault_name), header=None)
    df_final = df_initial.append(df_intermediate, ignore_index=True)  # Append df_initial & df_intermediate dataframe
    df_initial = df_final  # Swap value of df_final to df_initial dataframe

# Update for time bar
    pbar.update(n=1)
# Updating while loop
    start_flt_no = start_flt_no + 1  
    
# Export combined RSQSim output to CSV file
df_final.to_csv(r'/home/govindlinux/PhD/RSQSim_file_prep/%s/Combined_%d-%d_faults_eqs_output.csv'%(output_folder_name,start_flt_no1, end_flt_no), header=None)

# Close time bar
pbar.close()    

t: 5it [00:00,  8.06it/s]                                                                                               


### Part II -- END